<a href="https://colab.research.google.com/github/talpt/pyton/blob/main/BB_Cross_UP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Gerekli Kütüphanelerin İndirilmesi (Colab için gerekli kurulumlar)
try:
    import tvDatafeed
    import tradingview_screener
    # import colorama  # Kullanılmıyorsa kaldırılabilir
    import tqdm
    import mplfinance as mpf  # Mum grafiği için (isteğe bağlı)
except ImportError:
    # Colab için eksik kütüphaneleri yükleyin
    !pip install tradingview_screener
    !pip install git+https://github.com/rongardF/tvdatafeed  matplotlib openpyxl
    !pip install tradingview-screener==2.5.0  # Bazen sürüm çakışmaları olabiliyor
    # !pip install colorama  # Kullanılmıyorsa kaldırılabilir
    !pip install tqdm
    !pip install mplfinance  # Mum grafiği için (isteğe bağlı)

# from colorama import Fore, Style
from tradingview_screener import get_all_symbols
import numpy as np
from tvDatafeed import TvDatafeed, Interval
from tqdm import tqdm
import pandas as pd
import warnings
from datetime import datetime

# Uyarıları Engelle
warnings.filterwarnings("ignore", category=FutureWarning)

# TvDatafeed'in Çağırılması
tv = TvDatafeed()

def bollinger_yonu_belirle(data, band='SMA'):
    """
    Bollinger Bantlarının yönünü belirler.
    :param data: DataFrame (Bollinger Bantları hesaplanmış veri)
    :param band: 'SMA' (Orta Bant), 'Upper' (Üst Bant), 'Lower' (Alt Bant)
    :return: 'Yukarı', 'Aşağı', 'Yatay'
    """
    son_3_deger = data[band].iloc[-3:]  # Son 3 periyot
    if son_3_deger.is_monotonic_increasing:  # Artan
        return 'Yukarı'
    elif son_3_deger.is_monotonic_decreasing:  # Azalan
        return 'Aşağı'
    else:
        return 'Yatay'

# Kullanıcıdan zaman dilimi seçimi
print("Zaman Periyodu Seçin:")
print("1 - 4 Saatlik\n2 - Günlük\n3 - Haftalık\n4 - Aylık")
choice = input("Seçiminizi yapın (1/2/3/4): ")

if choice == "1":
    interval = Interval.in_4_hour
    interval_label = "4 Saatlik"
    n_bars = 300  # Geçmiş verileri de hesaba katmak için artırıyoruz
elif choice == "2":
    interval = Interval.in_daily
    interval_label = "Günlük"
    n_bars = 144
elif choice == "3":
    interval = Interval.in_weekly
    interval_label = "Haftalık"
    n_bars = 89
elif choice == "4":
    interval = Interval.in_monthly
    interval_label = "Aylık"
    n_bars = 40
else:
    print("Geçersiz seçim! Varsayılan olarak günlük veri kullanılacak.")
    interval = Interval.in_daily
    interval_label = "Günlük"
    n_bars = 100


# Tüm BIST Hisselerinin Listesini Alıyoruz
Hisseler = get_all_symbols(market='turkey')
Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
Hisseler = sorted(Hisseler)

# Sonuçları tutmak için bir DataFrame
tarama_sonuclari = pd.DataFrame(columns=['Hisse', 'Son Fiyat', 'Orta Bant', 'Uzaklık (%)'])

# Bollinger Bantları Hesaplama ve Tarama Fonksiyonu
def bollinger_tarama(data):
    # Bollinger Bantları Hesaplama (20 periyot, 2 standart sapma)
    data['SMA'] = data['Close'].rolling(window=20).mean()
    data['STD'] = data['Close'].rolling(window=20).std()
    data['Upper'] = data['SMA'] + (data['STD'] * 2)
    data['Lower'] = data['SMA'] - (data['STD'] * 2)

    # Son ve bir önceki mumun verileri
    son_kapanis = data['Close'].iloc[-1]
    onceki_kapanis = data['Close'].iloc[-2]
    orta_bant = data['SMA'].iloc[-1]
    onceki_orta_bant = data['SMA'].iloc[-2]
    ust_bant = data['Upper'].iloc[-1]

    # Bollinger Bantlarının Yönlerini Belirleme
    ust_bant_yonu = bollinger_yonu_belirle(data, band='Upper')
    orta_bant_yonu = bollinger_yonu_belirle(data, band='SMA')
    alt_bant_yonu = bollinger_yonu_belirle(data, band='Lower')

    # Koşulları Kontrol Et
    if (onceki_kapanis <= onceki_orta_bant) and (son_kapanis > orta_bant):
        # Uzaklık kontrolü (%0 - %5)
        uzaklik = ((son_kapanis - orta_bant) / orta_bant) * 100
        if 0 <= uzaklik <= 5:
            return son_kapanis, orta_bant, uzaklik, ust_bant_yonu, orta_bant_yonu, alt_bant_yonu
    return None, None, None, None, None, None


# Tarama İşlemi
for hisse in tqdm(Hisseler, desc="Tarama Yapılıyor", unit="hisse"):
    try:
        data = tv.get_hist(hisse, exchange='BIST', interval=interval, n_bars=n_bars)
        if data is None or data.empty:
            print(f"⚠ {hisse} hissesi için {interval_label} zaman aralığında veri bulunamadı.")
            continue

        # Sütun isimlerini kontrol et ve düzelt (GÜVENLİ YÖNTEM)
        for col in ['open', 'high', 'low', 'close', 'volume']:
            if col in data.columns:
                data.rename(columns={col: col.capitalize()}, inplace=True)
                # Alternatif: data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)

        # Bollinger Tarama Koşullarını Kontrol Etme
        son_fiyat, orta_bant, uzaklik, ust_bant_yonu, orta_bant_yonu, alt_bant_yonu = bollinger_tarama(data)
        if son_fiyat is not None:
            new_row = pd.DataFrame([{
                'Hisse': hisse,
                'Son Fiyat': son_fiyat,
                'Orta Bant': orta_bant,
                'Uzaklık (%)': uzaklik,
                'Üst Bant Yönü': ust_bant_yonu,  # Yeni eklenen sütun
                'Orta Bant Yönü': orta_bant_yonu,  # Yeni eklenen sütun
                'Alt Bant Yönü': alt_bant_yonu  # Yeni eklenen sütun
            }])
            tarama_sonuclari = pd.concat([tarama_sonuclari, new_row], ignore_index=True)


    except Exception as e:
        print(f"{hisse} için hata: {e}")  # Hata mesajı


# Tarama Sonuçlarını Görüntüleme ve Kaydetme
if not tarama_sonuclari.empty:
    print(f"\n{interval_label} periyotta, Bollinger Bandı Taramasında {len(tarama_sonuclari)} hisse bulundu.\n")

    # 1. Liste: Tüm Koşula Uyan Hisseler
    print("1. Liste: Bollinger Orta Bandın Üzerinde olup (Banda Uzaklık Yüzdesi %0 -%5 arasında) Koşula Uyan Hisseler")
    print(tarama_sonuclari.to_string(index=False))  # Daha okunabilir çıktı

    # 2. Liste: Özel Koşula Uyan Hisseler
    ozel_kosul = (
        (tarama_sonuclari['Üst Bant Yönü'] == 'Yukarı') &
        (tarama_sonuclari['Orta Bant Yönü'] == 'Yukarı') &
        (tarama_sonuclari['Alt Bant Yönü'] == 'Aşağı')
    )
    ozel_kosul_hisseler = tarama_sonuclari[ozel_kosul]

    if not ozel_kosul_hisseler.empty:
        print("\n2. Liste: Üst Band Yönü YUKARI/Orta Band Yönü YUKARI/Alt Band Yönü AŞAĞI Özel Koşuluna Uyan Hisseler")
        print(ozel_kosul_hisseler.to_string(index=False))
    else:
        print("\n2. Liste: Üst Band Yönü YUKARI/Orta Band Yönü YUKARI/Alt Band Yönü AŞAĞI Özel Koşuluna uyan hisse bulunamadı.")

    # Excel Dosyasına Kaydetme
    output_file = f"bollinger_tarama_{interval_label}_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.xlsx"
    with pd.ExcelWriter(output_file) as writer:
        tarama_sonuclari.to_excel(writer, sheet_name='Tüm Hisseler', index=False)
        if not ozel_kosul_hisseler.empty:
            ozel_kosul_hisseler.to_excel(writer, sheet_name='Özel Koşul Hisseler', index=False)
    print(f"\nSonuçlar '{output_file}' dosyasına kaydedildi.")
else:
    print(f"\n{interval_label} periyotta, Bollinger Bandı taramasına uygun hisse bulunamadı.")

Zaman Periyodu Seçin:
1 - 4 Saatlik
2 - Günlük
3 - Haftalık
4 - Aylık
Seçiminizi yapın (1/2/3/4): 3


Tarama Yapılıyor:   4%|▍         | 26/601 [00:14<05:43,  1.67hisse/s]ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol
Tarama Yapılıyor:   4%|▍         | 27/601 [00:14<04:57,  1.93hisse/s]

⚠ ALARK hissesi için Haftalık zaman aralığında veri bulunamadı.


Tarama Yapılıyor: 100%|██████████| 601/601 [05:29<00:00,  1.82hisse/s]


Haftalık periyotta, Bollinger Bandı Taramasında 4 hisse bulundu.

1. Liste: Bollinger Orta Bandın Üzerinde olup (Banda Uzaklık Yüzdesi %0 -%5 arasında) Koşula Uyan Hisseler
Hisse  Son Fiyat  Orta Bant  Uzaklık (%) Üst Bant Yönü Orta Bant Yönü Alt Bant Yönü
MEGMT      29.98    29.7150     0.891805         Aşağı          Aşağı         Aşağı
SDTTR     215.10   212.7700     1.095079         Aşağı          Aşağı         Yatay
YKSLN       7.69     7.3270     4.954279        Yukarı          Yatay         Aşağı
YUNSA       6.43     6.2225     3.334673         Yatay          Yatay         Yatay

2. Liste: Üst Band Yönü YUKARI/Orta Band Yönü YUKARI/Alt Band Yönü AŞAĞI Özel Koşuluna uyan hisse bulunamadı.

Sonuçlar 'bollinger_tarama_Haftalık_2025-03-20_19-17-49.xlsx' dosyasına kaydedildi.
